<a href="https://colab.research.google.com/github/metapat/Apprentissage-par-Renforcement-Profond/blob/main/notebooks/bonus-unit1/bonus-unit1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bonus Unité 1 : Apprenons à Huggy le chien 🐶 à rapporter un bâton

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit2/thumbnail.png" alt="Bonus Unit 1Thumbnail">

Dans ce cahier, nous consoliderons les acquis de la première unité en **apprenant à Huggy le chien à rapporter le bâton, puis en jouant directement avec lui dans votre navigateur**.

⬇️ Voici un exemple de ce que **vous serez capable de faire à la fin de cette unité**. ⬇️ (cliquez sur ▶ pour voir)


In [ ]:
%%html
<video controls autoplay><source src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy.mp4" type="video/mp4"></video>

### L'environnement 🎮
- Huggy le chien, un environnement créé par [Thomas Simonini](https://twitter.com/ThomasSimonini) based on [Puppo The Corgi](https://blog.unity.com/technology/puppo-the-corgi-cuteness-overload-with-the-unity-ml-agents-toolkit)

### La bibliothèque utilisée 📚

- [MLAgents](https://github.com/Unity-Technologies/ml-agents)

Nous essayons constamment d'améliorer nos tutoriels, donc **si vous trouvez des problèmes dans ce notebook**, veuillez [ouvrir un problème sur le dépôt Github](https://github.com/huggingface/deep-rl-class/issues).


## Objectifs de ce cahier 🏆

À la fin de ce cahier, vous serez capable de :
- Comprendre **l’espace d’états, l’espace d’actions et la fonction de récompense utilisés pour entraîner Huggy**.
- **Entraîner votre propre Huggy** à rapporter le bâton.
- Jouer **directement avec votre Huggy entraîné dans votre navigateur**.




## Ce carnet provient du cours d'apprentissage par renforcement profond.
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

Dans ce cours gratuit, vous allez :
- 📖 Étudier l’apprentissage par renforcement profond, tant sur le plan théorique que pratique.
- 🧑‍💻 Apprendre à utiliser des bibliothèques d’apprentissage par renforcement profond reconnues, telles que Stable Baselines3, RL Baselines3 Zoo, CleanRL et Sample Factory 2.0.
- 🤖 Entraîner des agents dans des environnements uniques.

Pour en savoir plus, consultez le programme. 👉 https://simoninithomas.github.io/deep-rl-course

Don’t forget to **<a href="http://eepurl.com/ic5ZUD">sign up to the course</a>** (we are collecting your email to be able to **send you the links when each Unit is published and give you information about the challenges and updates).**

Le meilleur moyen de rester en contact est de rejoindre notre serveur Discord pour échanger avec la communauté et avec nous 👉🏻 https://discord.gg/ydHrjt3WP5

## Prérequis 🏗️
Avant de commencer, vous devez :   
🔲 📚 **Comprendre les fondements de l’apprentissage par renforcement** (module critique, dépendance temporelle, hypothèse des récompenses…) en réalisant l’unité 1.   
🔲 📚 **Lire l’introduction de Huggy** en réalisant l’unité bonus 1.


## Set the GPU 💪
- To **accelerate the agent's training, we'll use a GPU**. To do that, go to `Runtime > Change Runtime type`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Step 1">

- `Hardware Accelerator > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2">

## Cloner le dépôt 🔽
- Nous devons cloner le dépôt contenant les **agents ML**.


In [ ]:
%%capture
# Clone the repository (can take 3min)
!git clone --depth 1 https://github.com/Unity-Technologies/ml-agents

## Configuration de l'environnement virtuel 🔽
- Pour que **ML-Agents** fonctionne correctement dans Colab, la version de Python utilisée par Colab doit respecter les exigences de la bibliothèque.
- Vous pouvez vérifier la version de Python requise grâce au paramètre `python_requires` dans les fichiers `setup.py`. Ces fichiers sont nécessaires à la configuration de la bibliothèque **ML-Agents** et se trouvent aux emplacements suivants :
  - `/content/ml-agents/ml-agents/setup.py`
  - `/content/ml-agents/ml-agents-envs/setup.py`
- La version Python actuelle de Colab (vérifiable avec `!python --version`) ne correspond pas au paramètre `python_requires` de la bibliothèque. Par conséquent, l'installation peut échouer silencieusement et entraîner des erreurs comme celles-ci lors de l'exécution ultérieure des mêmes commandes :
  - `/bin/bash: line 1: mlagents-learn: command not found`
  - `/bin/bash: line 1: mlagents-push-to-hf: command not found`

- Pour résoudre ce problème, nous allons créer un environnement virtuel avec une version de Python compatible avec la bibliothèque **ML-Agents**.

Remarque : *Pour assurer la compatibilité future, vérifiez toujours le paramètre `python_requires` dans les fichiers d'installation et configurez votre environnement virtuel avec la version maximale de Python prise en charge dans le script ci-dessous si la version de Python de Colab n'est pas compatible.*


In [ ]:
# Colab's Current Python Version (Incompatible with ML-Agents)
!python --version

In [ ]:
# Install virtualenv and create a virtual environment
!pip install virtualenv
!virtualenv myenv

# Download and install Miniconda
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# Activate Miniconda and install Python ver 3.10.12
!source /usr/local/bin/activate
!conda install -q -y --prefix /usr/local python=3.10.12 ujson  # Specify the version here

# Set environment variables for Python and conda paths
!export PYTHONPATH=/usr/local/lib/python3.10/site-packages/
!export CONDA_PREFIX=/usr/local/envs/myenv

In [ ]:
# Python Version in New Virtual Environment (Compatible with ML-Agents)
!python --version

## Installation des dépendances 🔽

In [ ]:
%%capture
# Go inside the repository and install the package (can take 3min)
%cd ml-agents
!pip3 install -e ./ml-agents-envs
!pip3 install -e ./ml-agents

### Télécharger et déplacer le fichier zip de l'environnement dans
`./trained-envs-executables/linux/`

- Notre fichier exécutable d'environnement se trouve dans une archive zip.
- Il faut le télécharger et le placer dans le répertoire approprié.
 `./trained-envs-executables/linux/`

In [ ]:
!mkdir ./trained-envs-executables
!mkdir ./trained-envs-executables/linux

Nous avons téléchargé le fichier Huggy.zip depuis https://github.com/huggingface/Huggy en utilisant `wget`


In [ ]:
!wget "https://github.com/huggingface/Huggy/raw/main/Huggy.zip" -O ./trained-envs-executables/linux/Huggy.zip

In [ ]:
%%capture
!unzip -d ./trained-envs-executables/linux/ ./trained-envs-executables/linux/Huggy.zip

Assurez-vous que votre fichier est accessible.

In [ ]:
!chmod -R 755 ./trained-envs-executables/linux/Huggy

## Récapitulons le fonctionnement de cet environnement
### L'espace d'états : ce que Huggy « perçoit ». Huggy ne « voit » pas son environnement. Nous lui fournissons plutôt des informations à son sujet :
- La position de la cible (le bâton)
- Sa position relative par rapport à la cible
- L'orientation de ses jambes.

Grâce à ces informations, Huggy **peut décider de la prochaine action à entreprendre pour atteindre son objectif**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy.jpg" alt="Huggy" width="100%">

### L'espace d'action : les mouvements que Huggy peut faire
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy-action.jpg" alt="Huggy action" width="100%">

Les jambes de Huggy sont actionnées par des moteurs articulaires. Cela signifie que pour atteindre la cible, Huggy doit apprendre à faire pivoter correctement les moteurs articulaires de chacune de ses jambes afin de pouvoir se déplacer.

### La fonction de récompense

La fonction de récompense est conçue pour que **Huggy atteigne son objectif** : rapporter le bâton.

Rappelons que l’un des fondements de l’apprentissage par renforcement est l’*hypothèse de récompense* : un objectif peut être décrit comme la **maximisation de la récompense cumulée attendue**.

Ici, notre objectif est que Huggy **se dirige vers le bâton sans trop tourner sur lui-même**. Par conséquent, notre fonction de récompense doit traduire cet objectif. Notre fonction de récompense :

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/reward.jpg" alt="Huggy reward function" width="100%">

- *Bonus d'orientation* : nous le **récompensons lorsqu'il s'approche de la cible.**
- *Pénalité de temps* : une pénalité de temps fixe est appliquée à chaque action afin de **l'obliger à atteindre le bâton le plus rapidement possible**.
- *Pénalité de rotation* : nous pénalisons Huggy **s'il tourne trop vite sur lui-même.**
- *Récompense pour avoir atteint la cible* : nous récompensons Huggy **lorsqu'il atteint la cible.**

## Créez le fichier de configuration Huggy.
- Dans ML-Agents, vous définissez les **hyperparamètres d'entraînement** dans des fichiers config.yaml.
- Dans le cadre de ce notebook, nous n'allons pas modifier les hyperparamètres. Toutefois, si vous souhaitez faire des essais, vous pouvez modifier d'autres hyperparamètres. Unity fournit une documentation très complète expliquant chacun d'eux [disponible ici](https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Training-Configuration-File.md).

- Il nous faut créer un fichier de configuration pour Huggy.
  - Pour ce faire, cliquez sur l'icône du dossier à gauche de votre écran.

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/create_file.png" alt="Create file" width="10%">

  - Allez à `/content/ml-agents/config/ppo`
  - Faites un clic droit et créez un nouveau fichier appelé `Huggy.yaml`

  <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/create-huggy.png" alt="Create huggy.yaml" width="20%">

- Copiez et collez le contenu ci-dessous 🔽

In [ ]:
behaviors:
  Huggy:
    trainer_type: ppo
    hyperparameters:
      batch_size: 2048
      buffer_size: 20480
      learning_rate: 0.0003
      beta: 0.005
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
      learning_rate_schedule: linear
    network_settings:
      normalize: true
      hidden_units: 512
      num_layers: 3
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.995
        strength: 1.0
    checkpoint_interval: 200000
    keep_checkpoints: 15
    max_steps: 2e6
    time_horizon: 1000
    summary_freq: 50000

- Don't forget to save the file!

- **In the case you want to modify the hyperparameters**, in Google Colab notebook, you can click here to open the config.yaml: `/content/ml-agents/config/ppo/Huggy.yaml`

- For instance **if you want to save more models during the training** (for now, we save every 200,000 training timesteps). You need to modify:
  - `checkpoint_interval`: The number of training timesteps collected between each checkpoint.
  - `keep_checkpoints`: The maximum number of model checkpoints to keep.

=> Just keep in mind that **decreasing the `checkpoint_interval` means more models to upload to the Hub and so a longer uploading time**
We’re now ready to train our agent 🔥.

## Train our agent

To train our agent, we just need to **launch mlagents-learn and select the executable containing the environment.**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/mllearn.png" alt="ml learn function" width="100%">

With ML Agents, we run a training script. We define four parameters:

1. `mlagents-learn <config>`: the path where the hyperparameter config file is.
2. `--env`: where the environment executable is.
3. `--run-id`: the name you want to give to your training run id.
4. `--no-graphics`: to not launch the visualization during the training.

Train the model and use the `--resume` flag to continue training in case of interruption.

> It will fail first time when you use `--resume`, try running the block again to bypass the error.



The training will take 30 to 45min depending on your machine (don't forget to **set up a GPU**), go take a ☕️you deserve it 🤗.

In [ ]:
!mlagents-learn ./config/ppo/Huggy.yaml --env=./trained-envs-executables/linux/Huggy/Huggy --run-id="Huggy2" --no-graphics

## Push the agent to the 🤗 Hub

- Now that we trained our agent, we’re **ready to push it to the Hub to be able to play with Huggy on your browser🔥.**

To be able to share your model with the community there are three more steps to follow:

1️⃣ (If it's not already done) create an account to HF ➡ https://huggingface.co/join

2️⃣ Sign in and then, you need to store your authentication token from the Hugging Face website.
- Create a new token (https://huggingface.co/settings/tokens) **with write role**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Create HF Token">

- Copy the token
- Run the cell below and paste the token

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

If you don't want to use a Google Colab or a Jupyter Notebook, you need to use this command instead: `huggingface-cli login`

Then, we simply need to run `mlagents-push-to-hf`.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/mlpush.png" alt="ml learn function" width="100%">

And we define 4 parameters:

1. `--run-id`: the name of the training run id.
2. `--local-dir`: where the agent was saved, it’s results/<run_id name>, so in my case results/First Training.
3. `--repo-id`: the name of the Hugging Face repo you want to create or update. It’s always <your huggingface username>/<the repo name>
If the repo does not exist **it will be created automatically**
4. `--commit-message`: since HF repos are git repository you need to define a commit message.

In [ ]:
!mlagents-push-to-hf --run-id="HuggyTraining" --local-dir="./results/Huggy2" --repo-id="ThomasSimonini/ppo-Huggy" --commit-message="Huggy"

Else, if everything worked you should have this at the end of the process(but with a different url 😆) :



```
Your model is pushed to the hub. You can view your model here: https://huggingface.co/ThomasSimonini/ppo-Huggy
```

It’s the link to your model repository. The repository contains a model card that explains how to use the model, your Tensorboard logs and your config file. **What’s awesome is that it’s a git repository, which means you can have different commits, update your repository with a new push, open Pull Requests, etc.**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/modelcard.png" alt="ml learn function" width="100%">

But now comes the best: **being able to play with Huggy online 👀.**

## Play with your Huggy 🐕

This step is the simplest:

- Open the game Huggy in your browser: https://huggingface.co/spaces/ThomasSimonini/Huggy

- Click on Play with my Huggy model

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/load-huggy.jpg" alt="load-huggy" width="100%">

1. In step 1, type your username (your username is case sensitive: for instance, my username is ThomasSimonini not thomassimonini or ThOmasImoNInI) and click on the search button.

2. In step 2, select your model repository.

3. In step 3, **choose which model you want to replay**:
  - I have multiple ones, since we saved a model every 500000 timesteps.
  - But since I want the more recent, I choose `Huggy.onnx`

👉 What’s nice **is to try with different models steps to see the improvement of the agent.**

Congrats on finishing this bonus unit!

You can now sit and enjoy playing with your Huggy 🐶. And don't **forget to spread the love by sharing Huggy with your friends 🤗**. And if you share about it on social media, **please tag us @huggingface and me @simoninithomas**

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit-bonus1/huggy-cover.jpeg" alt="Huggy cover" width="100%">


## Keep Learning, Stay  awesome 🤗